# データベースをバックエンドに持つメッセージメモリをエージェントに追加する

このノートブックでは、外部のメッセージストアを使用するメモリをエージェントに追加する方法を説明します。このノートブックを進める前に、以下のノートブックを確認してください。これらのノートブックを基にして構築します。

- [LLMチェーンにメモリを追加する](adding_memory.ipynb)
- [カスタムエージェント](../../agents/examples/custom_agent.ipynb)
- [メモリ付きエージェント](agetn_with_memory.ipynb)

外部メッセージストアを持つメモリをエージェントに追加するために、以下の手順を行います。

1. メッセージを格納するための外部データベースに接続する`RedisChatMessageHistory`を作成します。
2. そのチャット履歴をメモリとして使用して`LLMChain`を作成します。
3. その`LLMChain`を使用してカスタムエージェントを作成します。

この演習の目的のために、検索ツールにアクセスできるシンプルなカスタムエージェントを作成し、`ConversationBufferMemory`クラスを利用します。

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSearchAPIWrapper
import os

redis_password = os.environ.get("REDIS_PASSWORD")

In [ ]:
redis_password = os.environ.get("REDIS_PASSWORD")

In [ ]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

`chat_history`変数の使用に注意してください。これは、PromptTemplate内で使用されており、ConversationBufferMemory内の動的キー名と一致しています。

In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can in Japanese. When searching, first search in Japanese, and if you cannot find the answer anywhere, consider searching in English as well. Be sure to answer in Japanese. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

ここでデータベースよりバックされたChatMessageHistoryを作成します。

In [ ]:
message_history = RedisChatMessageHistory(url='redis://redis-13197.c294.ap-northeast-1-2.ec2.cloud.redislabs.com:13197', ttl=60*60*24*10,password=redis_password, session_id='my-redis')
memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=message_history)

これで、Memoryオブジェクトを持つLLMChainを構築し、エージェントを作成することができます。

In [ ]:
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0, model_name="gpt-4"), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_chain.run(input="Dead by Daylightの最新のアップデートはいつ行われましたか？")

このエージェントの記憶力を試すために、以前のやりとりの情報に依存して正しく回答されるような追質問をすることができます。

In [ ]:
agent_chain.run(input="恵みのトーテムの現在の効果を教えてください。")

In [ ]:
message_history.messages